In [ ]:
!pip install sense2vec
!pip install -U wn==0.0.22

In [ ]:
# first off all we imported libraries which we need
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
!pip install transformers

In [ ]:
!pip install spacytextblob
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm

In [ ]:
# Load sense2vec with comments from reddit
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz

In [ ]:
# unzip and extract the tar file to reveal the contents of the tar file
!tar -xvf s2v_reddit_2015_md.tar.gz

In [ ]:
# load the sense2vec vectors using the comments from reddit
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk('s2v_old')

In [ ]:
f1=open('QandA.txt')
q=[]
lines=f1.read().lower().split('\n')
for i in lines[:-1]:
  j=i.split("','")
  j[0]=j[0][1:]
  j[1]=j[1][:-1]
  q.append(j)

In [ ]:
q

In [ ]:
answers=[i[1] for i in q]

In [ ]:
qa=[]
for i in q:
  if i[1] not in i[0]:
    qa.append(i)

In [ ]:
qa

In [ ]:
distractors={}
for i in answers:
  distractors[i]=[]

In [ ]:
f2=open('keywords.txt')
keywords=[]
for i in f2.read().lower().split('\n')[:-1]:
  keywords.append(i[1:-1])

In [ ]:
keywords

Distractor from Sense2Vec

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()

In [ ]:
for answer in answers:
  sent=''
  if s2v.get_best_sense(answer)==None:
    for i in answer.split():
      sense = s2v.get_best_sense(i)
      if sense!=None:
        most_similar = s2v.most_similar(sense,n=20)
        if most_similar!=None:
          for k in most_similar:
            #print(k,lemmatizer.lemmatize(k[0][:k[0].index('|')]),lemmatizer.lemmatize(i),k[0][:k[0].index('|')],i,ps.stem(i),ps.stem(k[0][:k[0].index('|')]))
            if lemmatizer.lemmatize(k[0][:k[0].index('|')])!=lemmatizer.lemmatize(i) and k[0][:k[0].index('|')]!=i and ps.stem(i)!=ps.stem(k[0][:k[0].index('|')]):
              break
          most_similar = k[0][:k[0].index('|')]
          sent=sent+most_similar+'_'
        else:
          sent=sent+i+'_'
      else:
        sent=sent+i+'_'
  else:
      sense = s2v.get_best_sense(answer)
      most_similar = s2v.most_similar(sense,n=20)
      if most_similar!=None:
        for k in most_similar:
          #print('k:',k,'lem-k:',lemmatizer.lemmatize(k[0][:k[0].index('|')]),'lem-i:',lemmatizer.lemmatize(i),k[0][:k[0].index('|')],i,ps.stem(i),ps.stem(k[0][:k[0].index('|')]))
          if lemmatizer.lemmatize(k[0][:k[0].index('|')])!=lemmatizer.lemmatize(answer) and k[0][:k[0].index('|')]!=answer and ps.stem(answer)!=ps.stem(k[0][:k[0].index('|')]):
            break
      sent = k[0][:k[0].index('|')]
  sent=sent.replace('_',' ').strip()
  distractors[answer].append(sent)
  print(answer,'-',sent)   

Distractor by using most similar target word

In [ ]:
import os
import urllib.request
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
import gensim
import gensim.downloader as api
dataset = api.load("text8")
data = [d for d in dataset]

In [ ]:
# run below till the saving path if for 1st time

In [ ]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_for_training = list(tagged_document(data))

In [ ]:
modelGensim = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)

In [ ]:
modelGensim.build_vocab(data_for_training)

In [ ]:
#training model 
modelGensim.train(data_for_training, total_examples=modelGensim.corpus_count, epochs=modelGensim.epochs)

In [ ]:
print ("Saving model")
save_path_model = 'GensimModel'
modelGensim.save(save_path_model)

In [ ]:
#END OF RUNNING

In [ ]:
#LOAD TRAINED MODEL FROM DRIVE

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
modelGensim=Doc2Vec.load('GensimModel')


In [ ]:
def Sort_Tuple(tup): 
  
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    tup.sort(key = lambda x: x[1]) 
    return tup 

In [ ]:
similar={}
for i in answers:
  print('\n',i)
  o=[]
  for j in keywords:
      x=i.split(' ')
      y=j.split(' ')
      #print(x,y)
      #x1=list(map(lambda h: emmbed_dict[h[0]],x))
      #y1=list(map(lambda h: emmbed_dict[h[0]],y))
      #o.append((j[0],spatial.distance.euclidean(sum(x1),sum(y1))))
      o.append((j,spatial.distance.euclidean(modelGensim.infer_vector(x),modelGensim.infer_vector(y))))
  similar[i]=Sort_Tuple(o)
  #0 will be itself
  for k in similar[i]:
    #print(k)
    if k[0]!=i:
      print(k[0])
      distractors[i].append(k[0])
      break

Distractors using BERT

In [ ]:
from transformers import pipeline

In [ ]:
modelBert = pipeline('fill-mask', model='bert-base-uncased')

In [ ]:
f3=open('context.txt')

In [ ]:
context=[]
for i in f3.read().lower().split('\n')[:-1]:
  #print(i)
  j=i.split("','")
  #print(j)
  context.append((j[0][1:],j[1][:-1]))

In [ ]:
context

In [ ]:
def Sort_List(tup): 
    tup.sort(key = lambda x: x['score']) 
    return tup 

In [ ]:
extraDistractors={}
for i in answers:
  extraDistractors[i]=[]

In [ ]:
for i in context:
    c=i[0].lower()
    #print(keywords[i],':',c)
    print(c,i[1])
    c=c.replace(i[1].lower().strip(),'[MASK]',1)
    pred=modelBert(c)
    #print(c)
    #print(pred)
    l=Sort_List(pred)
    extraDistractors[i[1]]=[]
    flag=0
    for j in l:
      if j['token_str'] not in i[1] and i[1] not in j['token_str']:
        print(i[1],'-',j['token_str'])
        if flag==0:
          flag=1
          try:
            distractors[i[1]].append(j['token_str'])
          except:
            pass
        else:
          extraDistractors[i[1]].append(j['token_str'])

  


Unwanted Distractors

In [ ]:
extraDistractors

In [ ]:
distractors

In [ ]:
for i in distractors:
  c=0
  for j in range(len(distractors[i])):
    if c>=len(extraDistractors[i]):
      break
    if i in distractors[i][j] or distractors[i][j] in i:
      if i in extraDistractors[i][c] or extraDistractors[i][c] not in i:
        print(distractors[i][j],extraDistractors[i][c])
        distractors[i][j]=extraDistractors[i][c]
        break
      else:
        c+=1
      c+=1

In [ ]:
distractors

In [ ]:
d=distractors

In [ ]:
distractors=d

In [ ]:
for i in distractors:
  options=set()
  for j in distractors[i]:
    x=j.replace('-',' ').strip().lower()
    #print(x,' , ',i,' , ',ps.stem(i),' , ',ps.stem(x),' , ',lemmatizer.lemmatize(x),' , ',lemmatizer.lemmatize(i))
    if x!=i and ps.stem(i)!=ps.stem(x) and lemmatizer.lemmatize(x)!=lemmatizer.lemmatize(i):
      options.add(x)
  distractors[i]=options

In [ ]:
distractors

In [ ]:
import random

In [ ]:
f = open("MCQ.txt", "w")

In [ ]:
for i in qa:
  print('Q:',i[0])
  f.write('Q:'+i[0]+'\n')
  l=list(set(list(distractors[i[1]])+[i[1]]))
  random.shuffle(l)
  for j in range(1,len(l)+1):
    print(j,'.',l[j-1])
    f.write(str(j)+'.'+l[j-1]+'\n')
  print('\nA:',i[1],'\n')
  f.write('\nA:'+i[1]+'\n\n')
f.close()